In [1]:
cd ..

/home/is/akiyoshi-n/my-project


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
from datetime import datetime
import torch
import torch.nn as nn
from torch import cuda
from src.my_project.dataset import load_dataset, load_text_dataset
from src.my_project.train import ActClassifier
from transformers import BertTokenizer, BertJapaneseTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from src.my_project.dataset import preprocess_dataset, preprocess_multiclass_dataset

In [3]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs')
# パラメータの設定
MAX_LEN = 128 # 最大トークン数．本データでは最大トークン数が105なので，全て符号化できる．
# DROP_RATE = 0.4
BATCH_SIZE = 16 # バッチサイズ：分割した時の一つのデータ数．一回で学習するデータ数
NUM_EPOCHS = 100 # 何周学習させるか．エポック数
LEARNING_RATE = 2e-5 # 学習率
num_folds = 5
PATIENCE = 5 # 忍耐値．早期停止の役割を担う．モデル性能が改善されないエポック数が5を超えると訓練停止
SEED = 2023 # 乱数シード
num_labels = 2

# デバイスの指定
device = "cuda:1" if cuda.is_available() else "cpu"

timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

In [4]:
# データの読み込み
data = load_dataset(f"{DATASET_PATH}/act_classification_final.xlsx")
# 損失関数の定義
CRITERION = nn.CrossEntropyLoss()

/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [12]:
# 事前学習済みモデル（BERT）の指定
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# 交差検証の実行
trainer = ActClassifier(tokenizer=tokenizer, model_name = MODEL_NAME, criterion=CRITERION, device=device, seed=SEED)
fold_scores, fold_f1, best_model = trainer.cross_validation(MODEL_NAME, data, num_folds, NUM_EPOCHS, 
                                                            device, LEARNING_RATE, 
                                                            BATCH_SIZE, output_dir, MAX_LEN,timestamp)


-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.5863636363636363
macro f1: 0.5565989679076875
-----------------Fold: 2-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.6363636363636364
macro f1: 0.6339129711290457
-----------------Fold: 3-----------------


...

Early stopping. No improvement in loss.
Accuracy: 0.6681818181818182
macro f1: 0.665130632415188
-----------------Fold: 4-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.6727272727272727
macro f1: 0.6699999999999999
-----------------Fold: 5-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.6090909090909091
macro f1: 0.6064564439637241
-------------------------------------
finished epochs : [28, 31, 23, 24, 31]
Average Accuracy: 0.6345454545454545
Average macro f1: 0.626419803083129


In [10]:
# 東北大BERT-v2
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

trainer = ActClassifier(tokenizer=tokenizer, model_name = MODEL_NAME, criterion=CRITERION, device=device, seed=SEED)
fold_scores, fold_f1, best_model, all_predictions, all_labels = trainer.cross_validation(MODEL_NAME, data, num_folds, NUM_EPOCHS, 
                                                            device, LEARNING_RATE, 
                                                            BATCH_SIZE, output_dir, MAX_LEN, timestamp, num_labels)

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.7045454545454546
macro f1: 0.704246034208186
-----------------Fold: 2-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.7
macro f1: 0.6975
-----------------Fold: 3-----------------


...

Early stopping. No improvement in loss.
Accuracy: 0.7363636363636363
macro f1: 0.7263916287846299
-----------------Fold: 4-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.759090909090909
macro f1: 0.7554068511254222
-----------------Fold: 5-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.6409090909090909
macro f1: 0.638752052545156
-------------------------------------
finished epochs : [10, 12, 19, 13, 18]
Average Accuracy: 0.708181818181818
Average macro f1: 0.7044593133326789


In [14]:
print(all_predictions[:10])
print(all_labels[:10])

[1, 0, 1, 1, 1, 0, 0, 0, 1, 1]
[1, 0, 1, 1, 1, 0, 0, 0, 1, 1]


In [15]:
best_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [16]:
# 保存したモデルのディレクトリパス
model_path = f"{output_dir}/cl-tohoku/bert-base-japanese-v22023-11-21T20:15:32"

# モデルを読み込む
model = BertForSequenceClassification.from_pretrained(model_path)

MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:
'''
# 東北大BERT-v2
MODEL_NAME = 'cl-tohoku/bert-large-japanese-v2'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

trainer = ActClassifier(tokenizer=tokenizer, model_name = MODEL_NAME, criterion=CRITERION, device=device, seed=SEED)
fold_scores, fold_f1, best_model = trainer.cross_validation(MODEL_NAME, data, num_folds, NUM_EPOCHS, 
                                                            device, LEARNING_RATE, 
                                                            BATCH_SIZE, output_dir, MAX_LEN, timestamp)
'''

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-large-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.7318181818181818
macro f1: 0.729802868502675
-----------------Fold: 2-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-large-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.6772727272727272
macro f1: 0.6644540396554317
-----------------Fold: 3-----------------


...

Early stopping. No improvement in loss.
Accuracy: 0.7090909090909091
macro f1: 0.7055871528939445
-----------------Fold: 4-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-large-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.7272727272727273
macro f1: 0.724517906336088
-----------------Fold: 5-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-large-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.6636363636363637
macro f1: 0.6626326259946951
-------------------------------------
finished epochs : [48, 11, 12, 14, 12]
Average Accuracy: 0.7018181818181819
Average macro f1: 0.6973989186765668


In [5]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 交差検証の実行
trainer = ActClassifier(tokenizer=tokenizer, model_name = MODEL_NAME, criterion=CRITERION, device=device, seed=SEED)
fold_scores, fold_f1, best_model = trainer.cross_validation(MODEL_NAME, data, num_folds, NUM_EPOCHS, 
                                                            device, LEARNING_RATE, 
                                                            BATCH_SIZE, output_dir, MAX_LEN, timestamp)


TypeError: ActClassifier.cross_validation() missing 1 required positional argument: 'num_labels'

In [12]:
# 東北大BERT-v2，1100件全部のデータを用いて学習を行う．
# 東北大BERT-v2
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

trainer = ActClassifier(tokenizer=tokenizer, model_name = MODEL_NAME, criterion=CRITERION, device=device, seed=SEED)
model = trainer.train_model_all_data(MODEL_NAME, data, NUM_EPOCHS, device, LEARNING_RATE, BATCH_SIZE, output_dir, MAX_LEN, timestamp, num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
-------------------------------------
finished epochs : 13


In [14]:
'''
import pickle
# インスタンスをファイルに保存
save_path = f'{output_dir}/all_data_ActClassifier_tohoku_v2.pkl'
with open(save_path, 'wb') as output_file:
    pickle.dump(trainer, output_file)
'''

In [6]:
# 保存したインスタンスを読み取り
import pickle
load_path = f'{output_dir}/all_data_ActClassifier_tohoku_v2.pkl'
with open(load_path, 'rb') as input_file:
    trainer = pickle.load(input_file)

In [7]:
# 半教師あり学習を用いるためにmodelを用いて各テキストの確率を算出
# まずは，テストデータの読み込み
test_data = load_text_dataset('/home/is/akiyoshi-n/my-project/notebooks/data_extraction/add_data_sub.txt.xlsx')

In [8]:
probabilities, labels = trainer.prediction(test_data)

In [9]:
add_data = trainer.get_text_and_label(test_data=test_data, total_probabilities=probabilities, total_labels=labels, threshold=0.99)

In [19]:
add_data1 = add_data['texts']
add_data2 = add_data['texts']
add_data3 = add_data1 + add_data2

In [17]:
from collections import Counter
Counter(add_data['labels'])

Counter({1: 2243, 0: 1069})

In [16]:
# リストのadd_data['labels']の値の個数を確認
add_data['labels']

AttributeError: 'list' object has no attribute 'value_counts'

In [9]:
# 東北大BERT-v2
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
fold_scores, fold_f1, best_model, all_predictions, all_labels = trainer.Semi_Supervised_Learning_cross_validation(MODEL_NAME, data, add_data, num_folds, NUM_EPOCHS, 
                                                            device, LEARNING_RATE, 
                                                            BATCH_SIZE, output_dir, MAX_LEN, timestamp, num_labels)

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.7090909090909091
macro f1: 0.7049702455787444
-----------------Fold: 2-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.75
macro f1: 0.7435082140964494
-----------------Fold: 3-----------------


...

Early stopping. No improvement in loss.
Accuracy: 0.740909090909091
macro f1: 0.7261948950850455
-----------------Fold: 4-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.7136363636363636
macro f1: 0.7092572003943698
-----------------Fold: 5-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Early stopping. No improvement in loss.
Accuracy: 0.6727272727272727
macro f1: 0.67098703888335
-------------------------------------
finished epochs : [13, 10, 15, 20, 9]
Average Accuracy: 0.7172727272727274
Average macro f1: 0.7109835188075918
